In [1]:
from elasticsearch import Elasticsearch 
import util as ut
from elasticsearch import helpers

In [2]:
# Connect to the elastic cluster
es=Elasticsearch([{'host':'localhost','port':9200}])

In [5]:
es.indices.create(index='opal_dataset')
es.indices.create(index='opal_distribution')
es.indices.create(index='opal_measurements')

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'opal_measurements'}

# Indexing pipeline

In [8]:
offset=200000
num_of_sparql_query=1000

for _ in range(5): 
    
    print('Size of Dcat:Dataset {0}'.format(es.count(index='opal_dataset')['count']))
    
    
    sparqls=ut.generate_sparql_queries(offset=offset,num_query=num_of_sparql_query)
    offset+=num_of_sparql_query
    
    results=ut.submit_generated_sparql(sparqls)
    
    # number of queries must match
    assert len(results)==num_of_sparql_query
    
    
    try:
        document_index,dist_index,measures_index=ut.response_processer(results)
    except TypeError as tyerr:
        print('Response not successful')
        print("error: {0}".format(err))
        break
        
    
    
    helpers.bulk(es, ut.gendata('opal_dataset',document_index))
    helpers.bulk(es, ut.gendata('opal_distribution',dist_index))
    helpers.bulk(es, ut.gendata('opal_measurements',measures_index))

Size of Dcat:Dataset 200000
Size of Dcat:Dataset 200942
Size of Dcat:Dataset 202000
Size of Dcat:Dataset 202851
Size of Dcat:Dataset 203976


In [9]:
offset

205000

# NOTE

URIs should be shortened.

#### RequestError(400, 'action_request_validation_exception', 'Validation Failed: 1: id is too long, must be no longer than 512 bytes but was: 653;2